# Beartooth Least Cost Path Ultra Marathon

In [1]:
from btlcp.geo import locations

The following line will execute the `echo` function which will for each argument print the value and insert a line break.

In [3]:
peaks = locations()

peaks

frozenset({('Beartooth Mountain', (45.060833, -109.568611)),
           ('Castle Mountain', (45.098889, -109.630556)),
           ('Castle Rock Mountain', (45.098333, -109.653889)),
           ('Castle Rock Spire', (45.097222, -109.643056)),
           ('Granite Peak', (45.163427, -109.807456)),
           ('Mount Peal', (45.160833, -109.772778)),
           ('Mount Wood', (45.275278, -109.813333)),
           ('Silver Run Peak', (45.121111, -109.545278)),
           ('Tempest Mountain', (45.166944, -109.791944)),
           ('Whitetail Peak', (45.088889, -109.587778))})